## Введение в обработку естественного языка

## Урок 3. Embedding word2vec fasttext

Задача поиск похожих по эмбеддингам

Скачиваем датасет (источник): положительные, отрицательные.

или можно через ноутбук

!wget https://www.dropbox.com/s/fnpq3z4bcnoktiv/positive.csv
!wget https://www.dropbox.com/s/r6u59ljhhjdg6j0/negative.csv

что надо сделать
1. объединить в одну выборку
2. на основе word2vec/fasttext/glove/слоя Embedding реализовать метод поиска ближайших твитов
(на вход метода должен приходить запрос (какой-то твит, вопрос) и количество вариантов вывода к примеру 5-ть, ваш метод должен возвращать 5-ть ближайших твитов к этому запросу)
3. Проверить насколько хорошо работают подходы

In [29]:
import numpy as np

import pandas as pd

import string

from pymorphy2 import MorphAnalyzer

from stop_words import get_stop_words

import annoy

from gensim.models import Word2Vec, FastText

import pickle

from tqdm import tqdm_notebook

In [30]:
df_pos = pd.read_csv('positive.csv', sep=';', header=None, usecols=[3])

df_neg = pd.read_csv('negative.csv', sep=';', header=None, usecols=[3])

df = pd.concat([df_neg, df_pos])

df.columns=['text']

df

,text
0,на работе был полный пиддес :| и так каждое за...
1,"Коллеги сидят рубятся в Urban terror, а я из-з..."
2,@elina_4post как говорят обещаного три года жд...
3,"Желаю хорошего полёта и удачной посадки,я буду..."
4,"Обновил за каким-то лешим surf, теперь не рабо..."
...,...
114906,"Спала в родительском доме, на своей кровати......"
114907,RT @jebesilofyt: Эх... Мы немного решили сокра...
114908,"Что происходит со мной, когда в эфире #proacti..."
114909,"""Любимая,я подарю тебе эту звезду..."" Имя како..."


In [31]:
morpher = MorphAnalyzer()

stop_w = set(get_stop_words("ru"))

exclude = set(string.punctuation)

In [32]:
def preprocess_txt(line):
    spls = "".join(i for i in line.strip() if i not in exclude).split()
    spls = [morpher.parse(i.lower())[0].normal_form for i in spls]
    spls = [i for i in spls if i not in stop_w and i != ""]
    return spls

In [33]:
sentences = [ preprocess_txt(t) for t in tqdm_notebook(df['text']) ]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """Entry point for launching an IPython kernel.


  0%|          | 0/226834 [00:00<?, ?it/s]

In [34]:
def get_sentence_vector(model, sentence):
    r  = [model.wv[w] for w in sentence if w in model.wv ]
    
    return sum(r) / len(r) if r else np.zeros(300)

In [35]:
def build_index(model):
    index = annoy.AnnoyIndex(300 ,'angular')
    for i, snt in tqdm_notebook(enumerate(sentences)):
        v = get_sentence_vector(model, snt)
        index.add_item(i, v)
    index.build(10)
    
    return index

In [36]:
def find_nearest_tweets(model, index, tw, n=5):
    tw = preprocess_txt(tw)
    twv = get_sentence_vector(model, tw)
    nns = index.get_nns_by_vector(twv, n)
    
    return [df['text'].iloc[i] for i in nns]

In [37]:
def test_nearest_tweets(model, index, tw, n=5):
    print(50*'-')
    print(f"Nearest tweets for '{tw}':\n")
    ntws = find_nearest_tweets(model, index, tw, n)
    for tw in ntws:
        print(tw, '\n')

In [39]:
modelW2V = Word2Vec(sentences=sentences, size=300, window=5, min_count=1)

indexW2V = build_index(modelW2V)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


0it [00:00, ?it/s]

In [40]:
for i in np.random.randint(len(sentences), size=10):
    test_nearest_tweets(modelW2V, indexW2V, df['text'].iloc[i])

--------------------------------------------------
Nearest tweets for 'RT @vanilovy69: Надо делать уроки,но я играю в дракончиков,я не могу бросить их замок :( Когда я прекращу заниматься хуйней :(':

RT @vanilovy69: Надо делать уроки,но я играю в дракончиков,я не могу бросить их замок :( Когда я прекращу заниматься хуйней :( 

RT @Polina319: надо бы проверить, села ли @I_want_Cobain делать таблицу, но меня заставляют чистить клетку(
P.S. если ты меня слышишь, сцуч… 

RT @bywopyxawepy: ххх:Блин, у меня по лицу от косметики аллергические пятна пошли! =( ууу:Кошмар, и че делать 

RT @hapysugumis: ххх:Блин, у меня по лицу от косметики аллергические пятна пошли! =( ууу:Кошмар, и че делать 

RT @wanoqave: Это они меня учат как делать фото со свечами :)   Ха! Видели бы они, как я на старенький зенит с ручной выдержкой, без шт 

--------------------------------------------------
Nearest tweets for 'а ведь встала же! :) 
Всем доброго утра и хорошей недели,чтобы все сдали свои долги и зачеты! Ф

In [45]:
modelFT = FastText(sentences=sentences, size=300, window=5, min_count=1)

indexFT = build_index(modelFT)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


0it [00:00, ?it/s]

In [46]:
for i in np.random.randint(len(sentences), size=10):
    test_nearest_tweets(modelW2V, indexW2V, df['text'].iloc[i])

--------------------------------------------------
Nearest tweets for '@xumenen @beast_extra бля, не знал что все так плохо( Без примечаний такого уровня конечно совсем не то будет, особенно с Red Son например':

@xumenen @beast_extra бля, не знал что все так плохо( Без примечаний такого уровня конечно совсем не то будет, особенно с Red Son например 

Пиздец водить совсем разучился, и инструктор походу догадался, что правила я нихуя не знаю( 

Ох. Вы бы знали, какой фигней я занимаюсь((( 

@KupreevaNastya ахахха от бессонницы стирка носков отлично помогает))) ты уже знаешь)) 

@Billie_Cut99 ААЗАХЗААХАЗА :DD БЛ, У НЕГО ПРОБЛЕМЫ. СКА, ЭТО ПЛОХО :СС 

--------------------------------------------------
Nearest tweets for 'На улице снег и на рабочем столе снежинки падают =)':

На улице снег и на рабочем столе снежинки падают =) 

На улице снег и на рабочем столе снежинки падают =) 

На улице снег и на рабочем столе снежинки падают =) 

На улице снег и на рабочем столе снежинки падают =) 

Н

In [47]:
for i in np.random.randint(len(sentences), size=10):
    test_nearest_tweets(modelFT, indexFT, df['text'].iloc[i])

--------------------------------------------------
Nearest tweets for 'вспомнила какие красивые у тебя глаза ) http://t.co/ntEKuR6HTv':

вспомнила какие красивые у тебя глаза ) http://t.co/ntEKuR6HTv 

http://t.co/v5jnkrQLTt Я не ревную, просто ей без волос и с одним глазом лучше.) 

апоаоврлпо
Гарри, зачем ты такой красивый?
/чувствую себя фаплом :(/ http://t.co/QW7RvxNlBC 

Здесь просто красиво...и страшно...но красиво....у меня минималки( http://t.co/itolgbzQ0A 

@Oksana121277 ахахах...тут мы с тоюой страшно красивы:* http://t.co/VEbmojKLRq 

--------------------------------------------------
Nearest tweets for '- Вокруг красивых девушек вертятся не только парни. - Да? А что же еще? - Слухи, моя дорогая..много слухов..)))))':

- Вокруг красивых девушек вертятся не только парни. - Да? А что же еще? - Слухи, моя дорогая..много слухов..))))) 

красивые девочки жалуются на одиночество..что же тогда с некрасивыми :( 

Бред,это все у меня есть  парень,а не которые просто не хотят что бы о

In [52]:
qs = ['Отличное средство от', 'Идиотская идея']

for q in qs:
    test_nearest_tweets(modelW2V, indexW2V, q, 5)

--------------------------------------------------
Nearest tweets for 'Отличное средство от':

Отличный день) благодаря тебе♥ http://t.co/b1iiU8nCmR 

Отличный стеб! Только так;) 
"Ведьмы из Сугаррамурди"
http://t.co/V7MU8l3KWK 

Отличная тарелка:) http://t.co/SPRv9rIBeA #РетвитнуТвитВзаимно 

@tugarin_alex ну не правда,не надо тут!по мне так вполне отличный день) 

@VoteSex Это было бы отличным подъёбом, если бы у тебя был ник Сквозняк, а так не прокатит) 

--------------------------------------------------
Nearest tweets for 'Идиотская идея':

@s_l_o_t  прикол)когда будет эта идея воплощена в жизнь?) 

Ахаха..Блин..Вхаха..Мамина идея была:D http://t.co/GgXq9hKqf7 

@nastushka_d @OlehNaumko По какой идее?)) У него есть владелец)) 

не то) да, атмосфера там должна быть соответствующая: творчески-доверительная 

@continuumsl Я о том же,Стас.Красивое,особенное всегда доставляет удовольствие.Люблю такое.Если это вещь, то я непрем должна прикоснуться)) 



In [53]:
for q in qs:
    test_nearest_tweets(modelFT, indexFT, q, 5)

--------------------------------------------------
Nearest tweets for 'Отличное средство от':

У кого то сессичные. А у меня депрессичные ( 

моё ахиренное, классное, отличное, весёлое настроение пропало. :( 

Сегодня на прекрасное вальс:   ^_^ Приятных выходных! 

Сегодня на прекрасное вальс:   ^_^ Приятных выходных! 

спасибо вам, за эти весёлые и разнообразные школьные будни) 

--------------------------------------------------
Nearest tweets for 'Идиотская идея':

@Aleksis381381 @Z8SS Нет у нас армии,нет у нас полиции,, но есть люди, одетые в форму, похожую на военную и на полицейскую ( 

Такое Г это фоззи.Нихрена интересного.Но выпичка у них отменная:) 

у bigstar такое общежитие мелкое и грязное и антисанитария пати :( 

У кого-то будет бурная ночка. Только я и Тэдди Рузвельт. Точнее американский текст про него, который надо перевести на русский( 

@NovikovEM @loupivan ну сойдемся, что это проблема у многих людей) у нас футбольных тренеров и президентов полные кухни) 



**Word2vec и fasttext показали посредственные результаты на приведенных примерах- семантически близкие твиты не были подобраны. Вероятнее всего это было связано с приведенными примерами ('Отличное средство от' и 'Идиотская идея') или недостаточным разнообразием твитов в исходных наборах.**  